In [1]:
# Imports

In [2]:
import pandas as pd    # for reading data files
import h5py   # for writing as HDF files
import numpy as np
import time   # for measuring elapsed time
import os     # for joining paths
import random
from collections import defaultdict  # for dictionary of unique elements
# import tensorflow as tf
# import matplotlib.pyplot as plt
# %matplotlib inline

In [3]:
eps = 0.0000000001

#read unique chromosomes
filePath = '/home/amitkrkc/bucket/mESC_data'
chromosomeList='unique_chromosomes.txt'
file_chromosome = os.path.join(filePath,chromosomeList)
f = open(file_chromosome)
chr_list= []
for line in f:
    chromosome = line[:-1] # discard the last element (new-line character)
    if chromosome != 'chrX' and chromosome != 'chrY':
        chr_list.append(chromosome) 
f.close()
print chr_list

['chr1', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9']


In [4]:
# read TRIP data
class TRIP_data:
    def __init__(self, filePath= '/home/amitkrkc/bucket/mESC_data', 
                 tripFile='TRIPdata_mPGK.txt'):
        
        # read TRIP file
        df_pos_exp = pd.read_csv(os.path.join(filePath,tripFile), sep='\t',skiprows=1)
        
        # create dictionary of chromosomes
        self.all_pos = {}
        self.all_mu = {}
        for chromosome in chr_list:
            df_chr = df_pos_exp[df_pos_exp['chromosome'] == chromosome]
            self.all_pos[chromosome] = df_chr['position'].values
            c1 = df_chr['expression_counts_1']
            c2 = df_chr['expression_counts_2']
            n1 = df_chr['normalization_counts_1']
            n2 = df_chr['normalization_counts_2']
            self.all_mu[chromosome] = np.log(0.5*(c1/n1 + c2/n2).values + eps)
            
# read BED data
class BED_data:
    def __init__(self,filePath='/home/amitkrkc/bucket/mESC_data',
                 featureListFile='featureList.txt', 
                 feat_ind = [0,1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,6,6,6,6]):
        
        # feature indices and unique features
        self.feat_ind = feat_ind
        self.unique_feat = np.unique(self.feat_ind)
        self.dim_feat = len(self.unique_feat)   #dimension of unique features (=7)
        
        #read feature lists
        file_feature = os.path.join(filePath,featureListFile)
        f = open(file_feature)
        feature_list= []
        for line in f:
            feature_list.append(line[:-1]) # -1 for removing new-line character
        f.close()
        self.feature_list = feature_list
        
        # create dictionary of unique indices
        self.dict = defaultdict(list)
        for i in range(len(feat_ind)):
            unique_ind = feat_ind[i]
            self.dict[unique_ind].append(i)

        
        #load all BED files into a 'big' data frame
        col_names = 'chrom\tchromStart\tchromEnd\tname\tscore\tstrand\tsignalValue\tpValue\tqValue'.split('\t')
        all_df  = pd.DataFrame(columns=col_names)
        for feat_idx in np.arange(len(feature_list), dtype=int):
            feat_file = feature_list[feat_idx]+'.bed'
            bed_file = os.path.join(filePath,feat_file)
            
            # read the BED file and append a new column FileIndex 
            # that corresponds to the index of the BED file
            df = pd.read_csv(bed_file, sep='\t', names=col_names, header=None)
            df['FileIndex'] = feat_idx
            
            # append to the big data frame
            all_df = all_df.append(df, ignore_index=True)

        # create dictionary of chromosomes
        self.all_df_dict = {}
        for chromosome in chr_list:
            all_chr = all_df[all_df['chrom']==chromosome]
            
            # sort by start time of chromosome
            all_chr.sort_values(by=['chromStart']) 
            
            self.all_df_dict[chromosome] = all_chr

    # computes average over the unique values of the feature indices
    def aggregate_features(self,feat):
        f1 = np.zeros(self.dim_feat)
        for kk,vv in self.dict.iteritems():
            f1[kk] = np.average(feat[vv])
        return f1
    
    # get all the features of the given chromosome at position pos_i (obtained from TRIP data)
    def position_query(self, pos_i, chromosome='chr2'):
        # select the data-frame corresponding to the chromosome
        all_chr = self.all_df_dict[chromosome]        
        
        # find indices in the data-frame where pos_i lies in the start-end of the chromosome
        ind = (all_chr.chromStart <= pos_i) & (all_chr.chromEnd >= pos_i)
        val = all_chr.loc[ind, 'signalValue'].values
        idx = all_chr.loc[ind, 'FileIndex'].values.astype(int)
        feat = np.zeros(len(self.feature_list))
        feat[idx] = val
        
        # aggreate 22-dimensional feat vector into 7-dimensional vector
        return self.aggregate_features(feat)
    
    # get an aggregated feature in a range
    def range_query(self, range_start, range_end, strategy='area_under_curve', chromosome='chr2'):
        # extract chromStart, chromEnd, signalValue and FileIndex columns as arrays from the data frame
        xx = self.all_df_dict[chromosome]
        all_chrom_start = xx.chromStart.values.astype(int)
        all_chrom_end = xx.chromEnd.values.astype(int)
        all_chrom_signal = xx.signalValue.values
        all_index = xx.FileIndex.values.astype(int)
        
        s1 = all_chrom_start
        e1 = all_chrom_end
        s2 = range_start
        e2 = range_end

        # compute overlaps
        ind_a =  (s2 >= s1) & (s2 <= e1) & (e2 >= s1) & (e2 <= e1)
        ind_d = (s2 >= s1) & (s2 <= e1) & (e2 >= e1)
        ind_b = (s1 >= s2) & (s1 <= e2) & (e1 >= s2) & (e1 <= e2)
        ind_c = (s1 >= s2) & (s1 <= e2) & (e1 >= e2)

        # extract signals
        signal = np.zeros(len(all_chrom_start))
        signal[ind_a] = all_chrom_signal[ind_a]
        signal[ind_b] = all_chrom_signal[ind_b]
        signal[ind_c] = all_chrom_signal[ind_c]
        signal[ind_d] = all_chrom_signal[ind_d]
        
        feat_22 = np.zeros(len(self.feat_ind))

        if strategy == 'area_under_curve' or strategy == 'sum_of_ranges':
            # compute area of the overlap 
            rect = np.zeros(len(all_chrom_start))
            rect[ind_a] = e2 - s2
            rect[ind_b] = e1[ind_b] - s1[ind_b]
            rect[ind_c] = e2 - s1[ind_c]
            rect[ind_d] = e1[ind_d] - s2
            
            if strategy=='area_under_curve':
                # compute area under the gate function defined by the BED file
                yy = signal*rect
            else: # sum of ranges
                yy = rect
            
            # aggregate  
            for i in range(len(self.feat_ind)):
                feat_22[i] = np.sum(yy[all_index == i])

        elif strategy=='max_signal':
            # aggregate the signals by their max
            for i in range(len(self.feat_ind)):
                feat_22[i] = np.max(signal[all_index == i])

        elif strategy=='mean_signal':
            # aggregate the signals by their average
            for i in range(len(self.feat_ind)):
                feat_22[i] = np.average(signal[all_index == i])
        else:
            raise InputError('Unsupported strategy. Supported strategies are: area_under_curve (default), sum_of_ranges, max_signal and mean_signal')
        
        return self.aggregate_features(feat_22)
    
    # performs range query in the interval [pos_i-delta, pos_i+delta]
    def delta_query(self, pos_i, delta=5000, chromosome='chr2', strategy='area_under_curve'):
        range_start = pos_i - delta
        range_end = pos_i + delta
        return self.range_query(range_start, range_end, strategy=strategy, chromosome=chromosome)

class Pairwise_data:
    def __init__(self, filePath='/home/amitkrkc/bucket/mESC_data'):
        # create dictionary of unique indices
        self.nij = defaultdict(list)
        for chromosome in chr_list: #chr_list is the list of chromosomes that is defined above
            nij_file = filePath+'/nij/nij.'+chromosome
            nij = pd.read_csv(nij_file, sep='\t', header=None)
            self.nij[chromosome] = nij
    
    # returns the (quantized index) pos_j and distance (dist_j) of num_position nearest neighbors of 
    # (quantized) index i.
    def get_neighbors(self, i, chromosome='chr2', num_position=100):
       
        # i-th column of nij
        nij = self.nij[chromosome]
        ww = nij[i].values

        # sort distances in ascending order, and only consider last num_position elements
        idx_sort = np.argsort(ww)
        idx_sort = idx_sort[-num_position:]

        dist_j = ww[idx_sort]
        pos_j = idx_sort
        return (pos_j, dist_j)

In [5]:
# perform range query for all rows of the nij matrix
def extract_features_all_rows(chromosome='chr2', strategy='area_under_curve'):
    num = nij.nij[chromosome].shape[1]
    feat_matrix = np.zeros((num, 7))
    for i in range(num):
        k = i + 1
        range_start = (k-1)*40000+1
        range_end = k*40000
        feat = bed_data.range_query(range_start, range_end, strategy=strategy,chromosome=chromosome)
        feat_matrix[i,:] = feat
    return feat_matrix

In [6]:
def process_all_positions(chromosome='chr2', num_position=100, delta=5000, strategy='area_under_curve'):
    # compute range query matrix
    feat_range_matrix = extract_features_all_rows(chromosome=chromosome, strategy=strategy)
    
    dim_feat = bed_data.dim_feat
    num_samples = len(trip_data.all_pos[chromosome])
    
    # all delta values for delta_query
    all_delta = np.arange(1000, 21000, 1000, dtype=int)
    
    tot_dim_feat = num_position + num_position*dim_feat + dim_feat + dim_feat*len(all_delta) # 807+ 7 (last 20*7  for delta-query)
    feat_matrix = np.zeros((num_samples, tot_dim_feat))
                           
    # get the expression level directly from the TRIP data
    mu_vector = trip_data.all_mu[chromosome]
             
    
    # iterate for all positions in the TRIP data
    for k in range(num_samples):        
        #get the k-th position from TRIP data
        pos_k = trip_data.all_pos[chromosome][k]
                         
        # self-feature for position pos_k
        feat_i = bed_data.position_query(pos_k, chromosome)
                     
        # delta query
        feat_delta = np.zeros(dim_feat*len(all_delta))
        
        for iter_delta in range(len(all_delta)):
            delta_i = all_delta[iter_delta]
            temp_feat = bed_data.delta_query(pos_k, delta=delta_i,chromosome=chromosome, strategy=strategy)
            feat_delta[iter_delta*dim_feat:(iter_delta+1)*dim_feat] = temp_feat
            
        # quantize pos_k and get 100 nearest neighbors
        i = 1+(pos_k-1)/40000
        (pos_j, dist_j) = nij.get_neighbors(i, chromosome=chromosome, num_position=num_position)
        
        feat_range_j = feat_range_matrix[pos_j,:].ravel()  
        feat_matrix[k,0:len(feat_range_j)] = feat_range_j
        
        # concatenate all features 
        feat_matrix[k,:] = np.concatenate((dist_j, feat_range_j, feat_i, feat_delta))
                           
    return feat_matrix, mu_vector

### read features from BED file and mean expression from TRIP file

In [ ]:
tic=time.time()
trip_data = TRIP_data() 
bed_data = BED_data()
nij = Pairwise_data()
toc=time.time()
print("Time taken: %f seconds"%(toc-tic))

Time taken: 47.060183 seconds


### create dataset 

In [ ]:
import h5py
for chromosome in chr_list:
    tic=time.time()
    feat_matrix, mu_vector = process_all_positions(chromosome=chromosome, 
                                                   num_position=100, 
                                                   delta=5000, 
                                                   strategy='area_under_curve')
    # save data
    h5_file = 'Features/feature_all_' + chromosome + '.h5'
    with h5py.File(h5_file,'w') as hf:
        hf.create_dataset('feature', data=feat_matrix)
        hf.create_dataset('expression', data=mu_vector)

    toc=time.time()
    print("Chromosome:" + chromosome +" Time taken: %f seconds"%(toc-tic))

In [ ]:
print nij.nij['chr1'].shape[1]